# RAG 파이프라인 예시 - LangChain + Chroma

본 노트북에서는 `laws/` 디렉토리에 있는 법령 JSON 파일들을 파싱하여,
1) 필요한 텍스트를 추출하고,
2) 청크 단위로 나눈 뒤,
3) LangChain의 임베딩 모델(여기서는 `HuggingFaceEmbeddings`)로 벡터를 만들고,
4) Chroma DB에 저장하는 과정을 시연합니다.


In [1]:
# !pip install "langchain==0.2.6"
# !pip install "ibm-watsonx-ai==1.0.10"
# !pip install "langchain_ibm==0.1.8"
# !pip install "langchain_community==0.2.6"
# !pip install "sentence-transformers==3.0.1"
# !pip install "chromadb==0.5.3"
# !pip install "pydantic==2.8.2"
# !pip install "langchain-huggingface==0.0.3"
# !pip install "python-dotenv==1.0.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 64.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Pr

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
sentence = ["확인용 임의 문장"]
emb = model.encode(sentence)[0]
print(len(emb))  # 768인지 확인

/home/ibmuser03/.venv/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


768


In [1]:
import os
import glob
import json

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [2]:
# 기타 유틸
def extract_text_from_law_json(json_data: dict) -> list:
    """
    하나의 법령 JSON에서 필요한 텍스트(문자열)만 추출하여 리스트로 반환.
    원하는 필드에 맞춰 자유롭게 수정 가능.
    """
    results = []

    law = json_data.get("법령", {})

    # (1) 개정문
    revision_obj = law.get("개정문", {}).get("개정문내용", [])
    for paragraph_list in revision_obj:  # 2차원 리스트로 가정
        for line in paragraph_list:
            if isinstance(line, str) and line.strip():
                results.append(line.strip())

    # (2) 조문
    provisions = law.get("조문", {}).get("조문단위", [])
    for provision in provisions:
        content = provision.get("조문내용", "")
        title = provision.get("조문제목", "")
        if title:
            results.append(f"[조문제목] {title}")
        if content:
            results.append(f"[조문내용] {content}")
        # 항/호 내부 텍스트도 추출
        if "항" in provision and "호" in provision["항"]:
            for ho_item in provision["항"]["호"]:
                ho_no = ho_item.get("호번호")
                ho_content = ho_item.get("호내용")
                if ho_no:
                    results.append(f"[호번호] {ho_no}")
                if ho_content:
                    results.append(f"[호내용] {ho_content}")

    # (3) 부칙
    addenda = law.get("부칙", {}).get("부칙단위", [])
    for addendum in addenda:
        add_text = addendum.get("부칙내용", [])
        for paragraph_list in add_text:
            for line in paragraph_list:
                if isinstance(line, str) and line.strip():
                    results.append(line.strip())

    # (4) 제개정이유 (필요하다면)
    reasons = law.get("제개정이유", {}).get("제개정이유내용", [])
    for paragraph_list in reasons:
        for line in paragraph_list:
            if isinstance(line, str) and line.strip():
                results.append(line.strip())

    return results

def chunk_text(text_list: list, max_chunk_size: int = 500) -> list:
    """
    길이가 긴 문장을 일정 크기(max_chunk_size)로 분할.
    여기서는 단순히 문자열 길이를 기준으로 쪼개는 예시입니다.
    """
    chunks = []
    for text in text_list:
        # text가 너무 길면 여러 청크로 나눔
        if len(text) <= max_chunk_size:
            chunks.append(text)
        else:
            start = 0
            while start < len(text):
                end = start + max_chunk_size
                chunks.append(text[start:end])
                start = end
    return chunks

In [3]:
# "laws/" 디렉토리에 있는 모든 json 파일을 읽고, Chroma에 저장

# (1) 사용할 임베딩 모델 설정 (LangChain)
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"  # 예시 모델명
)

/tmp/ipykernel_117717/3054589040.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(
/home/ibmuser03/.venv/lib64/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
# (2) JSON 파싱 & 청크 -> 전체 텍스트 리스트 구성
laws_dir = "laws"  # JSON 파일들이 있는 디렉토리
json_files = glob.glob(os.path.join(laws_dir, "*.json"))

all_docs = []       # 최종적으로 임베딩할 텍스트 목록
all_metadatas = []  # 메타데이터 목록 (파일명 등)

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
    
    # 텍스트 추출
    extracted_texts = extract_text_from_law_json(json_data)
    # 청크화
    chunks = chunk_text(extracted_texts, max_chunk_size=500)
    
    # all_docs 에 추가
    for chunk in chunks:
        all_docs.append(chunk)
        all_metadatas.append({
            "source_file": os.path.basename(file_path),
            "preview": chunk[:30]  # chunk 앞부분 30자 정도를 메타정보로
        })

In [5]:
# (3) Chroma 벡터 스토어 생성
# 이미 DB가 있다면 업데이트할 수도 있으나, 여기서는 새로 만듭니다.

persist_directory = "chroma_db"  # Chroma 데이터가 저장될 폴더

# all_docs 와 all_metadatas를 이용해 Chroma 인스턴스 생성
vectorstore = Chroma.from_texts(
    texts=all_docs,
    embedding=embedding_model,
    metadatas=all_metadatas,
    persist_directory=persist_directory
)
# 생성 후, DB를 디스크에 저장
vectorstore.persist()

print("Chroma 벡터DB에 데이터 적재 완료!")

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

위에서 생성한 Chroma DB(`chroma_db` 폴더)는 RAG 파이프라인에서 **검색**용으로 활용할 수 있습니다. 
예를 들어, 질의 시 LangChain을 통해 아래와 같이 **유사도 검색**이 가능합니다:

```python
from langchain.vectorstores import Chroma

# 이미 생성된 DB 불러오기
vectorstore = Chroma(
    embedding_function=embedding_model, 
    persist_directory="chroma_db"
)

# 질의
query = "교통사고처리특례법상 형사합의 관련 내용 알려줘"
docs = vectorstore.similarity_search(query, k=3)
for i, doc in enumerate(docs, 1):
    print(f"\n[{i}] score=?")
    print(doc.metadata)  # 메타데이터(파일명, 미리보기)
    print(doc.page_content)  # 실제 텍스트
```

이렇게 검색된 텍스트들을 LLM에 주입하여 **추론**(답변)을 생성하면, RAG 워크플로우를 구축할 수 있습니다.